In [2]:
import torch
import torchvision
from random import shuffle
import gpytorch
from tqdm import tqdm
from gpytorch.models import ApproximateGP, ExactGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import UnwhitenedVariationalStrategy, VariationalStrategy
import matplotlib.pyplot as plt 


In [21]:
dataset = torchvision.datasets.mnist.MNIST("mnist", download=True, 
                                           transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))]))

In [22]:
targets = dataset.targets
class1_inds = torch.where(targets==0)[0]
class2_inds = torch.where(targets==4)[0]
inds = torch.cat([class1_inds, class2_inds])
inds = inds[torch.randperm(len(inds))]
targets = targets[inds]
targets = (targets > 0).int()
data = dataset.data[inds]
n_train = int(len(data)*0.2)
n_test = len(data) - n_train

In [23]:
data = data.reshape((-1, 784)).float()
std = data.std(dim=0)
std[std==0] = torch.ones_like(std[std==0])
data = (data - data.mean(dim=0))/std

In [24]:
train_x = data[:n_train].float()
test_x = data[n_train:].float()

train_y = targets[:n_train]
test_y = targets[n_train:]

In [25]:
N_INDUCING_POINTS = train_x.shape[0]

class GPClassificationModel(ApproximateGP):
    def __init__(self, train_x):
        variational_distribution = CholeskyVariationalDistribution(N_INDUCING_POINTS)
        variational_strategy = VariationalStrategy(
            self, train_x,
            variational_distribution, learn_inducing_locations=False
        )
        super(GPClassificationModel, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=784))

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        latent_pred = gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
        return latent_pred


model = GPClassificationModel(train_x)
likelihood = gpytorch.likelihoods.BernoulliLikelihood()

In [28]:

# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=784))
    
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
    
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)

In [29]:
dataset = torch.utils.data.TensorDataset(train_x, train_y)

In [11]:
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

# "Loss" for GPs - the marginal log likelihood
# num_data refers to the number of training datapoints
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
iters = 100

pbar = tqdm(range(iters))
for _ in pbar:

    # Zero backpropped gradients from previous iteration
    optimizer.zero_grad()
    # Get predictive output
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    optimizer.step()
    pbar.set_description(f"Loss: {loss.item()}")


Loss: 0.7257695198059082: 100%|██████████| 100/100 [10:24<00:00,  6.25s/it]


TypeError: rsub() received an invalid combination of arguments - got (Tensor, MultivariateNormal), but expected one of:
 * (Tensor input, Tensor other, *, Number alpha)
 * (Tensor input, Number other, Number alpha)


In [31]:
model.train()
likelihood.train()

GaussianLikelihood(
  (noise_covar): HomoskedasticNoise(
    (raw_noise_constraint): GreaterThan(1.000E-04)
  )
)

In [35]:
model.zero_grad()
num_parametrs = sum([p.numel() for p in model.parameters()])

hessian = torch.zeros(num_parametrs, num_parametrs).cpu()
model.zero_grad()
output = model(train_x)
loss = -mll(output, train_y)

grad_list = torch.autograd.grad(loss, model.parameters(), create_graph=True)
grad_i = torch.cat([g.view(-1) for g in grad_list]).cpu()
for i in range(0, num_parametrs):
    hessian[i] = torch.cat(
        [g.view(-1) for g in torch.autograd.grad(grad_i[i], model.parameters(), create_graph=True,
)]).cpu()
hessian_evals[batch_ind] = np.linalg.eigvalsh(hessian.detach().cpu().numpy())
del hessian

RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.

In [36]:
grad_list

(None,
 tensor([-0.5906]),
 tensor(0.1245, grad_fn=<SoftplusBackwardBackward>),
 tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+0

In [12]:
model.eval()
likelihood.eval()
output = likelihood(model(test_x)).mean
print(((output - test_y)**2).float().mean())


tensor(0.2500, grad_fn=<MeanBackward0>)


In [13]:
# model.train()
# likelihood.train()

# # Use the adam optimizer
# optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

# # "Loss" for GPs - the marginal log likelihood
# # num_data refers to the number of training datapoints
# mll = gpytorch.mlls.VariationalELBO(likelihood, model, train_y.numel())
# batch_size = 1000
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
# epochs = 10

# for _ in range(epochs):
#     model.train()
#     likelihood.train()
#     pbar = tqdm(enumerate(dataloader))
#     for i, (x, y) in pbar:
#         # Zero backpropped gradients from previous iteration
#         optimizer.zero_grad()
#         # Get predictive output
#         output = model(x)
#         # Calc loss and backprop gradients
#         loss = -mll(output, y)
#         loss.backward()
#         optimizer.step()
#         pbar.set_description(f"Loss: {loss.item()}")
#     model.eval()
#     likelihood.eval()
#     output = likelihood(model(test_x)).probs
#     preds = (output > 0.5).int()
#     print((preds == test_y).int().float().mean().item())


In [14]:
output = likelihood(model(test_x)).mean


In [15]:
output[:100]

tensor([0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994,
        0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994,
        0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994,
        0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994,
        0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994,
        0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994,
        0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994,
        0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994,
        0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994,
        0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994,
        0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994, 0.4994,
        0.4994], grad_fn=<SliceBackward>)

In [141]:
list(model.parameters())

[Parameter containing:
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]], requires_grad=True),
 Parameter containing:
 tensor([ 0.1368, -0.1761,  0.1023, -0.0733,  0.0717,  0.0811,  0.0878,  0.0732,
         -0.1124, -0.1154, -0.1324,  0.0951, -0.1370,  0.0974, -0.2112, -0.1434,
          0.1032, -0.1013, -0.1157,  0.0930,  0.0974, -0.0584, -0.0919, -0.1239,
          0.1106, -0.1379, -0.0738, -0.1274, -0.0807, -0.1034,  0.0933, -0.0647,
         -0.0940,  0.0871, -0.2157,  0.1238, -0.1177, -0.0557,  0.0701,  0.0805,
          0.1446, -0.0841, -0.0334,  0.0900,  0.0997, -0.0858, -0.0940, -0.1104,
          0.1083,  0.1334, -0.1071, -0.0880, -0.0764, -0.0512,  0.1084, -0.2025,
         -0.1365, -0.0738,  0.1198, -0.0875,  0.0600, -0.1236,  0.0944,  0.1248,
          0.1111, -0.100